In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

# 데이터 로딩 및 전처리
df = pd.read_csv('abalone.csv')
df = df.drop(columns=['id'])
df['Sex'] = LabelEncoder().fit_transform(df['Sex'])

X = df.drop(columns=['Rings']).values
y = df['Rings'].values

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Dataset 정의
class AbaloneRegressionDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_loader = DataLoader(AbaloneRegressionDataset(X_train, y_train), batch_size=64, shuffle=True)
test_loader = DataLoader(AbaloneRegressionDataset(X_test, y_test), batch_size=64)

# 모델 정의
class RegressionModel(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.net(x)

# 학습 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RegressionModel(X.shape[1]).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 학습 루프
for epoch in range(30):
    model.train()
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        preds = model(X_batch)
        loss = criterion(preds, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"[회귀] Epoch {epoch+1}/30, Loss: {loss.item():.4f}")

# 평가
model.eval()
total_loss = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        preds = model(X_batch)
        loss = criterion(preds, y_batch)
        total_loss += loss.item()
print(f"📉 회귀 테스트 MSE: {total_loss / len(test_loader):.4f}")


[회귀] Epoch 1/30, Loss: 92.1000
[회귀] Epoch 2/30, Loss: 16.3599
[회귀] Epoch 3/30, Loss: 10.8723
[회귀] Epoch 4/30, Loss: 5.6329
[회귀] Epoch 5/30, Loss: 14.7606
[회귀] Epoch 6/30, Loss: 7.0882
[회귀] Epoch 7/30, Loss: 9.4789
[회귀] Epoch 8/30, Loss: 8.1348
[회귀] Epoch 9/30, Loss: 7.9676
[회귀] Epoch 10/30, Loss: 1.4609
[회귀] Epoch 11/30, Loss: 1.6812
[회귀] Epoch 12/30, Loss: 4.3500
[회귀] Epoch 13/30, Loss: 10.0338
[회귀] Epoch 14/30, Loss: 3.0697
[회귀] Epoch 15/30, Loss: 3.4923
[회귀] Epoch 16/30, Loss: 1.7492
[회귀] Epoch 17/30, Loss: 2.7792
[회귀] Epoch 18/30, Loss: 2.9407
[회귀] Epoch 19/30, Loss: 3.5213
[회귀] Epoch 20/30, Loss: 4.6017
[회귀] Epoch 21/30, Loss: 6.9430
[회귀] Epoch 22/30, Loss: 4.6217
[회귀] Epoch 23/30, Loss: 4.1230
[회귀] Epoch 24/30, Loss: 11.1416
[회귀] Epoch 25/30, Loss: 6.1292
[회귀] Epoch 26/30, Loss: 4.2181
[회귀] Epoch 27/30, Loss: 1.6235
[회귀] Epoch 28/30, Loss: 4.4075
[회귀] Epoch 29/30, Loss: 5.3039
[회귀] Epoch 30/30, Loss: 5.4785
📉 회귀 테스트 MSE: 4.5365
